In [9]:
:e ImportQualifiedPost
:e FlexibleContexts
:e BlockArguments
:e TupleSections
:e FlexibleContexts
:e OverloadedStrings
:e LambdaCase

:l Plotting.hs
:l ../src/Control/Monad/Bayes/Class.hs
:l ../src/Control/Monad/Bayes/Enumerator.hs
:l ../src/Control/Monad/Bayes/Sampler.hs

import Control.Monad
import Data.List
import Data.Ord
import Control.Arrow (first)
import Data.Text (pack, Text)


# Sampling

Before discussing inference, we should understand how to sample from models. This notebook explains how to do that.

We'll start with a very simple model, namely:

In [10]:
model :: MonadSample m => m Bool
model = bernoulli 0.7

To take a sample, do:

In [11]:
sampleIO model

True

Or with a fixed seed:

In [12]:
sampleIOfixed model

True

To take multiple samples, you could rerun `sampleIO` many times, but it's somewhat more in the probabilistic programming spirit to instead define a distribution over multiple draws from `model` and then just sample once, as follows:

In [13]:
multipleDraws :: MonadSample m => m [Bool]
multipleDraws = replicateM 10 model

draws <- sampleIO multipleDraws

draws

[False,False,True,False,True,True,True,True,True,True]

We can write a function to convert a list of samples to an empirical distribution, like so:

In [14]:


emp = toEmpirical draws

emp



[("False",0.3),("True",0.7)]

In [15]:
plotVega emp

In fact, we could lean even further into the spirit of probabilistic programming, and transform `model` into a distribution over plots, and sample from that:

In [16]:
distributionOverPlots :: MonadSample m => m VegaLiteLab -- the type of plots
distributionOverPlots = plotVega . toEmpirical <$> replicateM 10 model

sampleIO distributionOverPlots


Now for a continuous distribution, consider

In [17]:
model2 :: MonadSample m => m Double
model2 = normal 0 1

Sampling is no different to before:

In [18]:
sampleIO model2

0.7460478609979049

And as before, to obtain multiple draws:

In [19]:
multipleDraws2 :: MonadSample m => m [Double]
multipleDraws2 = replicateM 10 model2

draws2 <- sampleIO multipleDraws2

draws2

[-0.6708814139321163,0.8786810061104819,-0.5654552019809386,-0.987910258131161,1.0494306065400019,0.2772218891357769,9.302143193597226e-2,-0.4077114177787587,-0.9101709804992575,-1.0691886268643593]

We'd like to view a histogram of samples, which in the limit of many samples should tend to the PDF of a normal distribution. Again, we could apply a histogram to the list of samples, but it's nicer to apply a `histogram` function to `multipleDraws`, to define a distribution over histograms from which we'll sample.

In [20]:
sampleIO $ plotVega . toEmpirical . toBins 0.05 <$> replicateM 100000 model2



In [21]:
model3 = do
    p <- bernoulli 0.7
    if p then normal 0 1 else normal 3 1

In [22]:
sampleIO $ plotVega . toEmpirical . toBins 0.05 <$> replicateM 100000 model3
